In [1]:
%load_ext cython
%load_ext autoreload
%autoreload 2

In [2]:
from psa import *
from MDAnalysis import Universe
from MDAnalysis.analysis.align import rotation_matrix
import numpy as np
import os, sys
WORKDIR = '/nfs/homes/sseyler/Repositories/python/psanalysis'
sys.path.append(WORKDIR)

In [3]:
print("Generating AdK CORE C-alpha reference coords + structure...")
cref_filename = '%s/structs/1ake_a_ca_core.pdb' % WORKDIR
oref_filename = '%s/structs/4ake_a_ca_core.pdb' % WORKDIR

c_ref = MDAnalysis.Universe(cref_filename)
o_ref = MDAnalysis.Universe(oref_filename)
u_ref = MDAnalysis.Universe(cref_filename)

c_ref_ca = c_ref.selectAtoms('name CA')
o_ref_ca = o_ref.selectAtoms('name CA')

adkCORE_resids = "(resid 1:29 or resid 60:121 or resid 160:214)"
c_ref_CORE_ca = c_ref_ca.selectAtoms(adkCORE_resids).coordinates() \
        - c_ref_ca.selectAtoms(adkCORE_resids).centerOfMass()
o_ref_CORE_ca = o_ref_ca.selectAtoms(adkCORE_resids).coordinates() \
        - o_ref_ca.selectAtoms(adkCORE_resids).centerOfMass()
ref_coords = 0.5*(c_ref_CORE_ca + o_ref_CORE_ca)

u_ref.atoms.translate(-c_ref_ca.selectAtoms(adkCORE_resids).centerOfMass())
o_ref.atoms.translate(-o_ref_ca.selectAtoms(adkCORE_resids).centerOfMass())
u_ref.selectAtoms(adkCORE_resids).CA.set_positions(ref_coords)

Generating AdK CORE C-alpha reference coords + structure...


In [142]:
print("Building collection of simulations...")
# List of method names (same as directory names)
# method_names = ['DIMS', 'FRODA', 'MAP']
method_names = ['DIMS', 'TMD', 'FRODA', 'MAP', 'iENM', 'MENM-SP', 'MENM-SD',       \
                'MDdMD', 'GOdMD', 'Morph', 'ANMP', 'LinInt']
labels = [] # Heat map labels
simulations = [] # List of simulation topology/trajectory filename pairs
universes = [] # List of MDAnalysis Universes representing simulations

# Build list of simulations, each represented by a pair of filenames
# ([topology filename], [trajectory filename]). Generate corresponding label
# list.
for method in method_names:
    # Note: DIMS uses the PSF topology format
    topname = 'top.psf' if ('DIMS' in method or 'TMD' in method) else 'top.pdb'
    pathname = 'path.dcd'
    method_dir = '{}/methods/{}'.format(WORKDIR, method)
    if method is not 'LinInt':
        if 'TMD' in method:
            for run in xrange(1, 11): # 3 runs per method
                run_dir = '{}/{:03n}'.format(method_dir, run)
                topology = '{}/{}'.format(method_dir, topname)
                trajectory = '{}/{}'.format(run_dir, pathname)
                labels.append(method + '(' + str(run) + ')')
                simulations.append((topology, trajectory))
#         nruns = 3 if 'TMD' not in method else 9
        else:
            for run in xrange(1, nruns+1): # 3 runs per method
                run_dir = '{}/{:03n}'.format(method_dir, run)
                topology = '{}/{}'.format(method_dir, topname)
                trajectory = '{}/{}'.format(run_dir, pathname)
                labels.append(method + '(' + str(run) + ')')
                simulations.append((topology, trajectory))
    else: # only one LinInt trajectory
        topology = '{}/{}'.format(method_dir, topname)
        trajectory = '{}/{}'.format(method_dir, pathname)
        labels.append(method)
        simulations.append((topology, trajectory))

Building collection of simulations...


In [143]:
# Generate simulation list represented as Universes. Each item, sim, in
# simulations is a topology/trajectory filename pair that is unpacked into
# an argument list with the "splat" ("*") operator.
for sim in simulations:
    universes.append(Universe(*sim))

In [144]:
print("Initializing Path Similarity Analysis...")
ref_selection = "name CA and " + adkCORE_resids
psa_full = PSA(universes, reference=u_ref, ref_select=ref_selection,
                    path_select="name CA", labels=labels)

Initializing Path Similarity Analysis...


In [145]:
print("Generating Path objects from aligned trajectories...")
psa_full.generate_paths(align=True, store=True)

Fitted frame   102/102  [100.0%]
Fitted frame    92/92  [100.0%]
Fitted frame    95/95  [100.0%]
Fitted frame    97/97  [100.0%]
Fitted frame    98/98  [100.0%]
Fitted frame    98/98  [100.0%]
Fitted frame    98/98  [100.0%]
Fitted frame    98/98  [100.0%]
Fitted frame    98/98  [100.0%]
Fitted frame    91/91  [100.0%]
Fitted frame    91/91  [100.0%]
Fitted frame    91/91  [100.0%]
Fitted frame   495/495  [100.0%]
Fitted frame   142/142  [100.0%]
Fitted frame   141/141  [100.0%]
Fitted frame   142/142  [100.0%]
Fitted frame   201/201  [100.0%]
Fitted frame   201/201  [100.0%]
Fitted frame   201/201  [100.0%]
Fitted frame    33/33  [100.0%]
Fitted frame    34/34  [100.0%]
Fitted frame    36/36  [100.0%]
Fitted frame    52/52  [100.0%]
Fitted frame    51/51  [100.0%]
Fitted frame    51/51  [100.0%]
Fitted frame    49/49  [100.0%]
Fitted frame    45/45  [100.0%]
Fitted frame    43/43  [100.0%]
Fitted frame    54/54  [100.0%]
Fitted frame    64/64  [100.0%]
Fitted frame    48/48  [100.0%]


Generating Path objects from aligned trajectories...


# =============================

In [146]:
# metric = 'discrete_frechet'
metric = frechet
linkage = 'ward' # 'single' 'complete' 'weighted' 'average'
plotname = 'df_ward_psa-withTMD-1-10.pdf'

In [147]:
print("Calculating distance matrix...")
psa_full.run(metric=metric)

Calculating distance matrix...


In [148]:
print("Plotting heat map-dendrogram for hierarchical clustering...")
psa_full.plot(filename=plotname, linkage=linkage);

Plotting heat map-dendrogram for hierarchical clustering...


In [149]:
df_default = psa_full.D

In [150]:
# metric = 'hausdorff'
metric = hausdorff
linkage = 'ward' # 'single' 'complete' 'weighted' 'average'
plotname = 'dh_ward_psa-withTMD-1-10.pdf'

In [151]:
print("Calculating distance matrix...")
psa_full.run(metric=metric)

Calculating distance matrix...


In [152]:
print("Plotting heat map-dendrogram for hierarchical clustering...")
psa_full.plot(filename=plotname, linkage=linkage);

Plotting heat map-dendrogram for hierarchical clustering...


In [153]:
dh_default = psa_full.D

# =============================

# Hausdorff

In [106]:
%%cython -f -c=-Ofast -c=-march=native
import numpy as np
cimport numpy as np
from libc.math cimport sqrt, fmax, fmin
import cython
cimport cython

ctypedef float[::1] (*minfcn_ptr)(float[:,::1])
ctypedef float (*maxfcn_ptr)(float[::1])

@cython.boundscheck(False)
@cython.wraparound(False)
cdef float[::1] getmin(float[:,::1] d):
    cdef np.intp_t i, j
    cdef float[::1] d_row = np.empty(d.shape[1], dtype='float32')
    cdef float[::1] rowmin = np.empty(d.shape[0], dtype='float32')
    cdef float m = 10000000
    
    for i in xrange(d.shape[0]):
        d_row = d[i,:]
        for j in xrange(d.shape[1]):
            m = fmin(d_row[j], m)
        rowmin[i] = m
    return rowmin

@cython.boundscheck(False)
@cython.wraparound(False)
cdef float getmax(float[::1] d):
    cdef np.intp_t i
    cdef float m = -1
    
    for i in xrange(d.shape[0]):
        m = fmax(d[i], m)
    return m

@cython.boundscheck(False)
@cython.wraparound(False)
def hausdorff(float[:,::1] P, float[:,::1] Q, np.intp_t N):
    cdef np.intp_t i, j, k, lenP, lenQ    
    lenP = P.shape[0]
    lenQ = Q.shape[0]

    assert int(P.shape[1]) == int(3*N)
    assert P.shape[1] == Q.shape[1]
    
    cdef float[:,::1] d = np.empty((lenP, lenQ), dtype='float32')
    cdef float s, diff = 0.0
    cdef minfcn_ptr minf = &getmin
    cdef maxfcn_ptr maxf = &getmax
    
    for i in xrange(lenP):
        for j in xrange(lenQ):
            s = 0.0
            for k in xrange(P.shape[1]):
                diff = P[i,k] - Q[j,k]
                s += diff*diff
            d[i,j] = s

#     return sqrt( fmax( maxf(minf(d)), maxf(np.amin(d, axis=1)) ) / N  )
#     return sqrt( fmax( maxf(minf(d)), maxf(minf(d.T)) ) / N  )
    return sqrt( fmax( np.amax(np.amin(d, axis=0)), np.amax(np.amin(d, axis=1)) ) / N  )

# Frechet

In [95]:
%%cython -f -c=-O3 -c=-march=native -c=-funroll-loops -c=-ffast-math
import numpy as np
cimport numpy as np
from libc.math cimport fmin, fmax, sqrt
cimport cython

ctypedef float (*fcn_ptr)(float[:,::1], float[:,::1], np.intp_t, np.intp_t)

@cython.boundscheck(False)
@cython.wraparound(False)
cdef float c(float[:,::1] d, float[:,::1] ca, np.intp_t i, np.intp_t j):
    cdef np.intp_t im1, jm1
    im1 = i-1
    jm1 = j-1
    if ca[i,j] != -1 : return ca[i,j]
    if i > 0:
        if j > 0: ca[i,j] = fmax( fmin(fmin(c(d,ca,im1,j),c(d,ca,i,jm1)),c(d,ca,im1,jm1)), d[i,j] )
        else:     ca[i,j] = fmax( c(d,ca,im1,0), d[i,0] )
    elif j > 0:   ca[i,j] = fmax( c(d,ca,0,jm1), d[0,j] )
    else:         ca[i,j] = d[0,0]
    return        ca[i,j]


@cython.boundscheck(False)
@cython.wraparound(False)
def frechet(float[:,::1] P, float[:,::1] Q, np.intp_t N):
    cdef np.intp_t lenP, lenQ, i, j, k
    
    lenP = P.shape[0]
    lenQ = Q.shape[0]
    
    cdef float[:,::1] d = np.empty((lenP, lenQ), dtype='float32')
    cdef float[:,::1] ca = -np.ones((lenP, lenQ), dtype='float32')
    cdef float s, temp
    cdef fcn_ptr cf = &c
    
    for i in xrange(lenP):
        for j in xrange(lenQ):
            s = 0.0
            for k in xrange(P.shape[1]):
                temp = P[i,k] - Q[j,k]
                s += temp*temp
            d[i,j] = s

    return sqrt( c(d, ca, lenP-1, lenQ-1) / N )

# Average Frechet

## Maximum-link-length (original) coupling distance function

In [ ]:
def c(i, j):
    if ca[i,j] != -1 : return ca[i,j]
    if i > 0:
        if j > 0: ca[i,j] = max( min(c(i-1,j),c(i,j-1),c(i-1,j-1)), d[i,j] )
        else:     ca[i,j] = max( c(i-1,0), d[i,0] )
    elif j > 0:   ca[i,j] = max( c(0,j-1), d[0,j] )
    else:         ca[i,j] = d[0,0]
    return        ca[i,j]

## Average-link-length coupling distance function

In [ ]:
def c(i, j):
    if cd[i,j] != -1 : return ca[i,j]
    
    if i > 0:
        if j > 0:
            if c(i-1,j) < c(i,j-1):
                if c(i-1,j) < c(i-1,j-1):
                    cd[i,j] = c(i-1,j)
                    cl[i,j] = cl[i-1,j] + 1
            elif c(i,j-1) < c(i-1,j-1):
                cd[i,j] = c(i,j-1)
                cl[i,j] = cl[i,j-1] + 1
            else:
                cd[i,j] = c(i-1,j-1)
                cl[i,j] = cl[i-1,j-1] + 1
        else:
            cd[i,0] = c(i-1,0) + d[i,0]
            cl[i,0] = cl[i-1,0] + 1
    elif j > 0:
        cd[0,j] = c(0,j-1) + d[0,j]
        cl[0,j] = cl[0,j-1] + 1
    else:
        cd[0,0] = d[0,0]
        cl[0,0] = 1

    return ca[i,j]

In [95]:
%%cython -f -c=-O3 -c=-march=native -c=-funroll-loops -c=-ffast-math
import numpy as np
cimport numpy as np
from libc.math cimport fmin, fmax, sqrt
cimport cython

ctypedef float (*fcn_ptr)(float[:,::1], float[:,::1], np.intp_t, np.intp_t)

@cython.boundscheck(False)
@cython.wraparound(False)
cdef float[::1] c(float[:,::1] d, float[:,:,::1] cd, np.intp_t i, np.intp_t j):
    cdef np.intp_t im1, jm1
    im1 = i-1
    jm1 = j-1
    if ca[i,j] != -1 : return cd[i,j,:]
    if i > 0:
        if j > 0:
            ci1j = c(d,cd,im1,j)
            cij1 = c(d,cd,i,jm1)
            ci1j1 = c(d,cd,im1,jm1)
            if ci1j[0] < cij1[0]:
                if ci1j[0] < ci1j1[0]:
                    cd[i,j,0] = ci1j[0] + d[i,j]
                    cd[i,j,1] = ci1j[1] + 1
            elif cij1[0] < ci1j1[0]:
                cd[i,j,0] = cij1[0] + d[i,j]
                cd[i,j,1] = cij1[1] + 1
            else:
                cd[i,j,0] = ci1j1[0] + d[i,j]
                cd[i,j,1] = ci1j1[1] + 1
        else:
            cd[i,0] = c(d,cd,cl,im1,0) + d[i,0]
            cl[i,0] = cl[i-1,0] + 1
    elif j > 0:
        cd[0,j] =  c(d,cd,cl,0,jm1) +d[0,j]
        cl[0,j] = cl[0,j-1] + 1
    else:
        cd[0,0] = d[0,0]
        cl[0,0] = 1
    return cd[i,j]


@cython.boundscheck(False)
@cython.wraparound(False)
def frechet_avg(float[:,::1] P, float[:,::1] Q, np.intp_t N):
    cdef np.intp_t lenP, lenQ, i, j, k
    
    lenP = P.shape[0]
    lenQ = Q.shape[0]
    
    cdef float[:,::1] d = np.empty((lenP, lenQ), dtype='float32')
    cdef float[:,::1] cd = -np.ones((lenP, lenQ), dtype='float32')
    cdef float[:,::1] cl = -np.ones((lenP, lenQ), dtype='float32')
    cdef float s, temp
    cdef fcn_ptr cf = &c
    
    for i in xrange(lenP):
        for j in xrange(lenQ):
            s = 0.0
            for k in xrange(P.shape[1]):
                temp = P[i,k] - Q[j,k]
                s += temp*temp
            d[i,j] = s

    return sqrt( c(d, cd, cl, lenP-1, lenQ-1) / N )